# MNIST Classification
> Performance analysis for MNIST Classification on all hardware platforms

- toc: true 
- badges: true
- comments: true
- categories: [MNIST,Rooflines,Performance Prediction]
- image: images/mnist.png

In [1]:
#hide
import pandas as pd
import numpy as np
import altair as alt

W = 600
H = 480

pd.options.display.float_format = '{:20,.4f}'.format
pd.options.display.max_rows = 500 # this will set limit of columns to 500
#master_df.loc[master_df['NN_Topology'] =='MLP']

csv_path = "./data/cleaned_csv/backup.csv"

# Theoretical Analysis of MNIST

### Rooflines for All Hardware Platforms and CNNs

Combining application requirements with hardware platform characteristics can be leveraged for performance predictions using UCB’s roofline models. Using assumptions for where weights, activation tensors, and state of a neural network are stored, combined with the size of the datatypes used, allow us to derive the arithmetic intensity of a neural network during inference. Combined with the roofline for a given hardware platform, we can provide insight as to whether a neural network will be memory or compute bound and guidance for what is theoretically possible in regards to its throughput.

In [2]:
#hide_input

#first process the following csv's to get clean ready-to-plot csv's
%run scripts/script_load_save_data.py
clean_csv_rooflines(path_topologies='c:/Users/alinav/Documents/GitHub/Qutibench_Web/_notebooks/data/topology_details.csv',
                    path_hardware='c:/Users/alinav/Documents/GitHub/Qutibench_Web/_notebooks/data/peakPerfBandHardPlatf.csv')

#Now get the cleaned csv, and plot it as a Roofline
%run scripts/altair_plots.py
rooflines(dataframe=pd.read_csv("data/cleaned_csv/rooflines_hardware_neural_networks.csv"), 
          neural_network='mnist')

alt.LayerChart(...)

### Performance Prediction

The following heatmap shows the theoretical performance for the listed hardware platforms for MNIST classification. The metric used for the theoretical performance is input/second.
This plot shows that a combination between prunning and quantization outputs some of the best performance results.

In [3]:
#hide_input
path_csv = 'data/performance_predictions_imagenet_mnist_cifar_2.csv'
    ## Reading csv file and converting data to (Neural network, Platform, Value)
df = pd.read_csv(path_csv)

df1 = pd.DataFrame()
columns = (df.loc[:, df.columns!='hardw']).columns #select all columns except first
for column in columns:
    df_=pd.melt(df, id_vars=['hardw'], value_vars=column) #melt df1 into a df1 of 2 columns
    df1=pd.concat([df1,df_])
df1.columns= ['y','x','values'] #setting new column names
 #replace 0s for NaN values because with 0s the grid doesn't show up
df1['values'] = df1['values'].replace({ 0.0:np.nan})
df_mnist = dataframe_contains(input_df=df1, column='x', value='MLP')
 #Saving above dataframes to csv file
df_mnist.to_csv('c:/Users/alinav/Documents/GitHub/QutibenchWeb/_notebooks/data/cleaned_csv/performance_prediction_mnist.csv', index = False)

heatmap(df_mnist, 'pink', 'Performance prediction for MNIST')

alt.LayerChart(...)

# Experimental Data Analysis

### Overview of All Measurements for MNIST 

In this table, within the rows, we show the type of hardware platforms that we used for this task (for example FPGA or GPU) and then more specifically the exact name of the different hardware platforms. For each hardware platform, we list the sweep of specific deployment parameters (batch sizes, operating modes etc) that were used for the experimentation in separate columns. In the columns, we show CNN topologies. When a CNN topology was implemented on a given hardware platform, we show in the corresponding cell the precisions (quantization information) and the channel pruning scale. Otherwise, “na” indicates that the topology wasn’t executed on this specific hardware platform. Many combinations between topology and hardware platform are not supported by the vendors dedicated software environments. INTx depicts a fixed point integer representation with x bits. FPy represents a floating point representation with y bits, for example FP32 is singe precision floating point. Table follows below.

In [4]:
#hide
%run scripts/script_tables.py  #get table with the experiments overview
print(pd.read_csv('data/overview_experiments_mnist.csv').to_markdown())

|    | Hardware   | Platform         | MLP                                 | Batch/Stream/Thread                  |
|---:|:-----------|:-----------------|:------------------------------------|:-------------------------------------|
|  0 | FPGA       | ZCU102-DPU       | na                                  | [1,2,3,4,5,6,7,8]                    |
|  1 | FPGA       | ZCU104-DPU       | na                                  | [1,2,3,4,5,6,7,8]                    |
|  2 | FPGA       | Ultra96-DPU      | na                                  | [1,2,3,4,5,6,7,8]                    |
|  3 | FPGA       | ZCU104-FINN      | [INT2, INT4] * [100%,50%,25%,12.5%] | [1,2,4,8,16,32,64,128,256,512,10000] |
|  4 | FPGA       | ZCU104-BISMO     | [INT2, INT4] * [100%,50%,25%,12.5%] | [2,4,8,16,32,64,128]                 |
|  5 | GPU        | TX2-maxn         | [FP16, FP32] * [100%,50%,25%,12.5%] | [1,2,4,8,16,32,64,128]               |
|  6 | GPU        | TX2-maxp         | [FP16, FP32] * [100%,50%,25%,12.5

In [5]:
#hide_input
%run scripts/script_tables.py 
dataframes = csv_to_dataframe_multiindex(['data/overview_experiments_mnist_.csv'])
for dataframe in dataframes:   
       display(HTML(dataframe.to_html(index=False)))

In [6]:
#hide
master_df = pd.read_csv(csv_path)
is_maxp = lambda row: row.HWType != "GPU" or row["Op mode"].split(",")[0] == "maxp"
maxp_df = master_df[master_df.apply(is_maxp, axis=1)]
maxp_df["hw_quant_prun"] = maxp_df.apply(lambda r: "_".join([r.HWType, r.Precision, r.PruningFactor]), axis=1)
mlp_df = maxp_df[(maxp_df["NN_Topology"] == "MLP") & maxp_df['lat-comp'].notna()]
mlp_df["hw_quant_prun"] = mlp_df.apply(lambda r: "_".join([r.HWType, r.Precision, r.PruningFactor]), axis=1)
mlp_df["PruningFactor"] = mlp_df["PruningFactor"].str.strip("%").astype(float)
norm_by_group(mlp_df, "lat-comp", "NN_Topology");
mlp_df["quant_model"] = mlp_df.Precision + '_' + mlp_df.HWType

mnist_df= maxp_df[(maxp_df.NN_Topology == 'MLP') & maxp_df["top1 [%]"].notna()]
mnist_df.rename(columns={"top1 [%]": "top1"}, inplace=True)
    
#filling GOPS values 
mnist_df['GOPS'] = mnist_df.apply(lambda r: 0.020029 if r.PruningFactor == '100%' else 
                                          (0.00582 if r.PruningFactor == '50%' else 
                                           (0.001862 if r.PruningFactor == '25%' else 0.000669 )) , axis=1)
#fill in tp-system and tp-cmp
mnist_df['tp-system'] = mnist_df['GOPS'] * mnist_df['fps-system']
mnist_df['tp-comp'] = mnist_df['GOPS'] * mnist_df['fps-comp']
mnist_df['GOPS'] = mnist_df['GOPS'] * mnist_df['batch/thread/stream']
mnist_df.head(200)

C:\Users\alinav\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\alinav\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\alinav\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_ind

,NN_Topology,HWType,Precision,Op mode,batch/thread/stream,lat-comp,fps-system,fps-comp,tp-system,tp-comp,top1,top5 [%],Base_Pwr_W,Idle_Pwr_W,Full_Pwr_W,GOPS,PruningFactor,level,hw_quant_prun
135,MLP,NCS,FP16,na,1,2.8926,218.8930,345.7130,4.3842,6.9243,98.8600,nan,0.5300,1.2000,1.7940,0.0200,100%,l3,NCS_FP16_100%
136,MLP,NCS,FP16,na,2,4.6846,247.3080,426.9310,4.9533,8.5510,98.8600,nan,0.5300,1.2000,1.9490,0.0401,100%,l3,NCS_FP16_100%
137,MLP,NCS,FP16,na,4,8.1182,269.4910,492.7190,5.3976,9.8687,98.8600,nan,0.5300,1.2000,1.9460,0.0801,100%,l3,NCS_FP16_100%
138,MLP,NCS,FP16,na,8,14.8038,282.6860,540.4000,5.6619,10.8237,98.8600,nan,0.5300,1.2000,2.0840,0.1602,100%,l3,NCS_FP16_100%
139,MLP,NCS,FP16,na,16,27.4935,292.8480,581.9560,5.8655,11.6560,98.8600,nan,0.5300,1.2000,2.1360,0.3205,100%,l3,NCS_FP16_100%
140,MLP,NCS,FP16,na,32,53.0572,297.9700,603.1220,5.9680,12.0799,98.8600,nan,0.5300,1.2000,2.1350,0.6409,100%,l3,NCS_FP16_100%
141,MLP,NCS,FP16,na,64,104.4350,301.5140,612.8230,6.0390,12.2742,98.8600,nan,0.5300,1.2000,2.1860,1.2819,100%,l3,NCS_FP16_100%
142,MLP,NCS,FP16,na,128,206.0070,303.4340,621.3370,6.0775,12.4448,98.8600,nan,0.5300,1.2000,2.2120,2.5637,100%,l3,NCS_FP16_100%
161,MLP,GPU,FP16,maxp,1,0.6983,976.1680,"1,466.4100",19.5517,29.3707,97.3000,nan,1.8000,4.7000,9.8810,0.0200,100%,l3,GPU_FP16_100%
162,MLP,GPU,FP16,maxp,2,0.7400,"1,635.7800","2,785.5500",32.7630,55.7918,97.3000,nan,1.8000,4.7000,9.9030,0.0401,100%,l3,GPU_FP16_100%


### Line Plot

In [7]:
#hide_input
figa_df = mlp_df[(mlp_df["HWType"].isin(["NCS", "ZCU104-Bismo", "U96-Quadcore A53"]))]
figb_df = mlp_df[(mlp_df["HWType"].isin(["GPU", "ZCU104-FINN", "U96-Quadcore A53"]))]

fig25s = []
fig25_dfs = [figa_df, figb_df]
for df in fig25_dfs:
    sel = alt.selection_multi(fields=["hw_quant_prun"], bind="legend")
    fig25_dot = alt.Chart(df).mark_point().encode(
        x='lat-comp',
        y=alt.Y('fps-comp', scale=alt.Scale(type="log")),
        color=select_color(sel, 'hw_quant_prun:N'),
        tooltip=['fps-comp', 'lat-comp', 'HWType', 'batch/thread/stream'],
    )
    fig25_line = alt.Chart(df).mark_line().encode(
        x='lat-comp',
        y='fps-comp',
        color=select_color(sel, 'hw_quant_prun:N'),
        tooltip=['fps-comp', 'lat-comp', 'HWType', 'batch/thread/stream'],
    )

    fig = (fig25_dot+fig25_line).properties(
        title="Latency versus Performance for Pruned and Quantized MLP Variants",
        width=W/len(fig25_dfs),
        height=H,
    ).add_selection(sel).interactive()
    
    fig25s.append(fig)
    
alt.hconcat(*fig25s)

alt.HConcatChart(...)

### Boxplots

In [8]:
#hide_input
#%run scripts/altair_plots.py  #run the plot script if it wasn't previously run
boxplot(df=mlp_df, yaxis="lat-comp", title="Latency by Hardware/Framework and Pruning for MLP")

alt.FacetChart(...)

In [9]:
#hide_input
#%run scripts/altair_plots.py  #run the plot script if it wasn't previously run
boxplot(mlp_df, "fps-comp", "Throughput by Hardware/Framework and Pruning for MLP")

alt.FacetChart(...)

In [10]:
#hide_input
#%run scripts/altair_plots.py  #run the plot script if it wasn't previously run
boxplot(df=mlp_df, yaxis="Full_Pwr_W", title="Power Consumption by Hardware/Framework and Pruning for MLP")

alt.FacetChart(...)

### Pareto Graphs

The following pareto graph presents the accuracy versus performance in fps for all the Hardware Platforms across different Pruning and Quantization configurations. This provides insights into accuracy-based comparisons.

In [11]:
#hide_input
#%run scripts/altair_plots.py  #run the plot script if it wasn't previously run
pareto_graph(df= mnist_df, 
             groupcol= 'hw_quant_prun',
             xcol = 'fps-comp', 
             ycol= 'top1',
             W= W, 
             H= H, 
             title= "MNIST Cassification Design Space: Accuracy versus Performance")

alt.LayerChart(...)

In [12]:
#hide
mnist_df.to_csv('data/cleaned_csv/experimental_data_mnist.csv', index = False)
mnist_df.head(200)

,NN_Topology,HWType,Precision,Op mode,batch/thread/stream,lat-comp,fps-system,fps-comp,tp-system,tp-comp,top1,top5 [%],Base_Pwr_W,Idle_Pwr_W,Full_Pwr_W,GOPS,PruningFactor,level,hw_quant_prun
135,MLP,NCS,FP16,na,1,2.8926,218.8930,345.7130,4.3842,6.9243,98.8600,nan,0.5300,1.2000,1.7940,0.0200,100%,l3,NCS_FP16_100%
136,MLP,NCS,FP16,na,2,4.6846,247.3080,426.9310,4.9533,8.5510,98.8600,nan,0.5300,1.2000,1.9490,0.0401,100%,l3,NCS_FP16_100%
137,MLP,NCS,FP16,na,4,8.1182,269.4910,492.7190,5.3976,9.8687,98.8600,nan,0.5300,1.2000,1.9460,0.0801,100%,l3,NCS_FP16_100%
138,MLP,NCS,FP16,na,8,14.8038,282.6860,540.4000,5.6619,10.8237,98.8600,nan,0.5300,1.2000,2.0840,0.1602,100%,l3,NCS_FP16_100%
139,MLP,NCS,FP16,na,16,27.4935,292.8480,581.9560,5.8655,11.6560,98.8600,nan,0.5300,1.2000,2.1360,0.3205,100%,l3,NCS_FP16_100%
140,MLP,NCS,FP16,na,32,53.0572,297.9700,603.1220,5.9680,12.0799,98.8600,nan,0.5300,1.2000,2.1350,0.6409,100%,l3,NCS_FP16_100%
141,MLP,NCS,FP16,na,64,104.4350,301.5140,612.8230,6.0390,12.2742,98.8600,nan,0.5300,1.2000,2.1860,1.2819,100%,l3,NCS_FP16_100%
142,MLP,NCS,FP16,na,128,206.0070,303.4340,621.3370,6.0775,12.4448,98.8600,nan,0.5300,1.2000,2.2120,2.5637,100%,l3,NCS_FP16_100%
161,MLP,GPU,FP16,maxp,1,0.6983,976.1680,"1,466.4100",19.5517,29.3707,97.3000,nan,1.8000,4.7000,9.8810,0.0200,100%,l3,GPU_FP16_100%
162,MLP,GPU,FP16,maxp,2,0.7400,"1,635.7800","2,785.5500",32.7630,55.7918,97.3000,nan,1.8000,4.7000,9.9030,0.0401,100%,l3,GPU_FP16_100%
